In [13]:
from datetime import date, timedelta, datetime
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
from functions import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import csv
import numpy as np
import pandas as pd
import unidecode
from tqdm import tqdm
from fake_useragent import UserAgent  



# Khởi tạo fake UserAgent
ua = UserAgent()

# Cấu hình trình duyệt với fake User-Agent và proxy
options = Options()
user_agent = ua.random  # Lấy một User-Agent ngẫu nhiên
options.add_argument(f"user-agent={user_agent}")


# Mở trình duyệt với undetected-chromedriver
driver = uc.Chrome(options=options)

# Đi đến URL mục tiêu
scrape_url = 'https://www.tripadvisor.com/'
driver.get(scrape_url)

# Tạo thời gian trễ ngẫu nhiên giữa các thao tác
time.sleep(random.uniform(1, 2))
driver.maximize_window()

# Một vài hành động mô phỏng hành vi tự nhiên của người dùng
actions = ActionChains(driver)
actions.move_by_offset(100, 100).perform()  # Di chuyển chuột ngẫu nhiên
time.sleep(random.uniform(0.5, 1))
actions.click().perform()  # Nhấp chuột ngẫu nhiên
time.sleep(random.uniform(1, 2))

In [14]:
# Nhập thành phố
place = input("Nhập thành phố bạn cần lấy dữ liệu: ")
time.sleep(random.uniform(0.8, 2))  # Thời gian trễ ngẫu nhiên

# Tìm thanh search
search_bar_location = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[4]/div/div/div[2]/div/form/div[1]/div/input")

# Thêm thời gian trễ ngẫu nhiên giữa các hành động
time.sleep(random.uniform(0.8, 2))
search_bar_location.clear()

time.sleep(random.uniform(0.8, 2))
search_bar_location.send_keys(place)

# Di chuyển chuột đến thanh tìm kiếm để giả lập người dùng thật
actions = ActionChains(driver)
actions.move_to_element(search_bar_location).perform()
time.sleep(random.uniform(0.8, 2))

# Tìm liên kết "Hotels" và bấm vào
hotel_link = driver.find_element(By.XPATH, "//a[@class='xyffN _G B- _S _W _T c G_ wSSLS biKBZ eeawa' and contains(@href, '/Hotels-g298085-Da_Nang-Hotels.html')]")
time.sleep(random.uniform(2, 4))

# Di chuyển chuột đến phần tử link "Hotels"
actions.move_to_element(hotel_link).perform()
time.sleep(random.uniform(2, 4))  # Thời gian trễ trước khi nhấp vào

# Bấm vào link "Hotels"
hotel_link.click()
time.sleep(random.uniform(2, 4))
print("Đã nhấp vào liên kết 'Hotels'")

# Thời gian chờ sau khi nhấn
time.sleep(random.uniform(0.8, 2))

Đã nhấp vào liên kết 'Hotels'


In [3]:
def getURL():
    page_source = BeautifulSoup(driver.page_source, 'html.parser')  # Thêm 'html.parser' để đảm bảo parse đúng cách
    hotels = page_source.find_all('a', class_='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS')
    all_hotel_url = []

    
    for hotel in hotels:
        hotel_link = hotel.get('href')
        # Kiểm tra nếu không có dấu '#' trong URL
        if hotel_link and '#' not in hotel_link:
            hotel_url = 'https://www.tripadvisor.com' + hotel_link
            if hotel_url not in all_hotel_url:
                all_hotel_url.append(hotel_url)

    return all_hotel_url

getURL()


['https://www.tripadvisor.com/Hotel_Review-g298085-d17732168-Reviews-Minh_Boutique-Da_Nang.html?spAttributionToken=Mjc1NzgyODE',
 'https://www.tripadvisor.com/Profile/cricketter',
 'https://www.tripadvisor.com/ShowUserReviews-g298085-d17732168-r910143784-Minh_Boutique-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d302750-Reviews-Furama_Resort_Danang-Da_Nang.html',
 'https://www.tripadvisor.com/Profile/tramy_BRD',
 'https://www.tripadvisor.com/ShowUserReviews-g298085-d302750-r893313258-Furama_Resort_Danang-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d13813945-Reviews-HAIAN_Beach_Hotel_Spa-Da_Nang.html',
 'https://www.tripadvisor.com/Profile/tahag5253',
 'https://www.tripadvisor.com/ShowUserReviews-g298085-d13813945-r893660799-HAIAN_Beach_Hotel_Spa-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d11689349-Reviews-Risemount_Premier_Resort_Danang-Da_Nang.html',
 'https://www.tripadvisor.com/Profile/mooimois',
 'https://www.tripadv

In [4]:
URLs_all_page = []

def getUrlOnPages():
    input_page = int(input('How many pages you want to scrape: '))  # Số trang cần scrape

    for page in range(input_page):
        # Cuộn nhiều lần để đảm bảo tất cả nội dung đã tải ra
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Cuộn xuống cuối trang
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height

        # Sau khi cuộn hết trang, lấy URL từ trang hiện tại
        URLs_one_page = getURL()  
        # URLs_all_page.extend(URLs_one_page)  # Thêm URL của trang hiện tại vào danh sách tổng
        
        
        # Kiểm tra và chỉ thêm những URL chưa có trong danh sách
        for url in URLs_one_page:
            if 'Profile' not in url and "ShowUserReviews" not in url and url not in URLs_all_page:
                URLs_all_page.append(url)
                
        try:
            # Tìm nút "Next" và bấm vào để chuyển sang trang tiếp theo
            # next_button = driver.find_element(By.CLASS_NAME, 'xkSty')  
            # next_button.click()
            next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_url = next_button.get_attribute('href')  # Lấy URL của trang tiếp theo
            driver.get(next_page_url)  # Mở trang tiếp theo
            time.sleep(1)  # Đợi để trang tiếp theo tải xong
        except:
            print(f"Không tìm thấy nút 'next' ở trang {page + 1}. Dừng scrape.")
            break  # Nếu không tìm thấy nút next, thoát khỏi vòng lặp

    return URLs_all_page

# Gọi hàm
getUrlOnPages()


Không tìm thấy nút 'next' ở trang 90. Dừng scrape.


['https://www.tripadvisor.com/Hotel_Review-g298085-d17732168-Reviews-Minh_Boutique-Da_Nang.html?spAttributionToken=Mjc1NzgyODE',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d302750-Reviews-Furama_Resort_Danang-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d13813945-Reviews-HAIAN_Beach_Hotel_Spa-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d11689349-Reviews-Risemount_Premier_Resort_Danang-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d16891462-Reviews-Sala_Danang_Beach_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d12919657-Reviews-Hilton_Da_Nang-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d7932663-Reviews-Vanda_Hotel-Da_Nang.html?spAttributionToken=MjY5OTg5OTA',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d1732187-Reviews-Pullman_Danang_Beach_Resort-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d15153595-Reviews-TMS_Hotel_Da_Nang_Beach-Da_Nang.h

In [15]:
URLs_all_page = URLs_all_page[195:]
URLs_all_page

['https://www.tripadvisor.com/Hotel_Review-g298085-d14914612-Reviews-Danabeach_Villas-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d8307584-Reviews-Vivyanne_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d12520859-Reviews-Pi_House-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d12504564-Reviews-Epin_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d12034393-Reviews-Song_Da_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d6885637-Reviews-Thanh_Long_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d10166793-Reviews-Salsa_1_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d10693540-Reviews-Minh_An_Hotel-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d10822323-Reviews-Sunrise_Hotel_Da_Nang-Da_Nang.html',
 'https://www.tripadvisor.com/Hotel_Review-g298085-d3914414-Reviews-Van_Xuan_Hotel-Da_Nang.html',
 'https://www.t

In [16]:
len(URLs_all_page)

1679

In [17]:
with open(f"Hotel_Data/hotel_{remove_accents_and_spaces(place)}.csv", 'a', newline='', encoding='utf-8-sig') as file_output:
    headers = [
        "Tourist Attraction", "Hotel Name", "Address", "Address Link", "Hotel Website", 
        "Hotline", "Ranking", "Reviewers", "Rating", "Location rating", 
        "Cleanliness rating", "Service rating", "Value rating", "Comment", "Price", 
        "URL", "Description", "Award", "Hotel Class", "Hotel Style 1", 
        "Hotel Style 2", "Language Spoken", "Numbers of near Restaurants",
        "Near Restaurant 1", "Link Near Restaurant 1", "Near Restaurant 2", "Link Near Restaurant 2",
        "Near Restaurant 3", "Link Near Restaurant 3", "Near Restaurant 4", "Link Near Restaurant 4",
        "Near Attraction 1", "Link Near Attraction 1", "Near Attraction 2", "Link Near Attraction 2", 
        "Near Attraction 3", "Link Near Attraction 3", "Near Attraction 4", "Link Near Attraction 4",
        "Numbers of near Attractions", "Hotel Image"
    ]
    
    writer = csv.DictWriter(file_output, delimiter=",", lineterminator="\n", fieldnames=headers)
    writer.writeheader()  # Ghi tiêu đề vào file

    # Thiết lập thanh tiến độ
    with tqdm(total=len(URLs_all_page), desc="Crawling hotel data", dynamic_ncols=True, unit="hotel", leave=False) as pbar:
        for hotel_url in URLs_all_page:
            # Quá trình thu thập dữ liệu cho từng khách sạn
            driver.get(hotel_url)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
        
            # Lấy thông tin tên khách sạn
            name_div = soup.find('div', {'class': 'ZlaHM f k'})
            if name_div:
                name_loc = name_div.find_all('div')
                if name_loc:
                    h1_tag = name_loc[0].find('h1')
                    if h1_tag and h1_tag.get_text():
                        name = h1_tag.get_text().strip()  
            time.sleep(random.uniform(0.25, 0.28))
            
            
            # Lấy thông tin địa chỉ
            address = np.nan  
            address_div = soup.find('div', {'class': 'FhOgt H3 f u fRLPH'})
            if address_div:
                address_span = address_div.find('span', {'class': 'biGQs _P pZUbB KxBGd'})
                if address_span:
                    address = address_span.get_text().strip()
                    
            time.sleep(random.uniform(0.25, 0.28))
            
            
            
            address_link = np.nan
            address_link_div = soup.find('div', {'class': 'ADOxT o _S tAYns'})
            if address_link_div:
                img_tag = address_link_div.find('img')
                if img_tag:
                    address_link = img_tag.get('src', np.nan)
                else:
                    address_link = np.nan
                    
            time.sleep(random.uniform(0.25, 0.28))
            
            website = np.nan
            website_div = soup.find('div', {'class': 'eeVey S4 H3 f u UKvGa'})
            if website_div:
                website_a = website_div.find('a', {'class': 'BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS'})
                if website_a:
                    website = 'https://www.tripadvisor.com' + website_a.get('href', '')
                    
            time.sleep(random.uniform(0.25, 0.28))
            
            phone_number = np.nan
            phone_div = soup.find('div', {'class': 'eeVey S4 H3 f u LGJIs'})
            if phone_div:
                phone_a = phone_div.find('a', href=True)
                if phone_a:
                    phone_number = phone_a.get('href').replace('tel:', '')
                    
            time.sleep(random.uniform(0.25, 0.28))
            
            ranking = np.nan
            ranking_div = soup.find('div', class_='biGQs _P pZUbB KxBGd')
            if ranking_div:
                ranking = ranking_div.get_text()
                
                
                    
            time.sleep(random.uniform(0.25, 0.28))
            
            reviewers = np.nan
            reviewers_span = soup.find('span', class_='biGQs _P pZUbB KxBGd')
            if reviewers_span:
                reviewers = reviewers_span.get_text().replace("reviews", "")
                
            time.sleep(random.uniform(0.25, 0.28))
            
            ratings = np.nan
            ratings_span = soup.find('span', class_='kJyXc P')
            if ratings_span:
                ratings = ratings_span.get_text()
                
            time.sleep(random.uniform(0.25, 0.28))
            
            location_rating, cleanliness_rating, service_rating, value_rating = np.nan, np.nan, np.nan, np.nan
            rating_blocks = soup.find_all('div', class_='RZjkd')
            if rating_blocks:
                ratings_dict = {}
                for block in rating_blocks:
                    block_name = block.find('div', class_='yFXuQ o W q').get_text()
                    score = block.find('div', class_='biGQs _P fiohW biKBZ osNWb').get_text()
                    ratings_dict[block_name] = score
                location_rating = ratings_dict.get("Location", np.nan)
                cleanliness_rating = ratings_dict.get("Cleanliness", np.nan)
                service_rating = ratings_dict.get("Service", np.nan)
                value_rating = ratings_dict.get("Value", np.nan)
                
            time.sleep(random.uniform(0.25, 0.28))
            
            comments = np.nan
            comments_div = soup.find('div', class_='PtOPK')
            if comments_div:
                comments = comments_div.get_text()
                
            time.sleep(random.uniform(0.25, 0.28))
            
            # Lấy giá khách sạn
            price = np.nan  # Giá trị mặc định là NaN
            price_div = soup.find('div', {'class': 'TSGVz f'})
            if price_div:
                price_loc = price_div.find_all('div')
                if price_loc and price_loc[0].find('div'):  # Kiểm tra nếu tìm thấy thẻ div bên trong
                    price = price_loc[0].find('div').get_text().strip().replace("$", "")
                else:
                    price = np.nan  # Trường hợp không có thẻ div bên trong

                    
            time.sleep(random.uniform(0.25, 0.28))
                    
                    
            # Tìm thẻ chứa mô tả
            description = np.nan
            description_div = soup.find('div', {'class': 'fIrGe _T'})

            # Nếu tìm thấy thẻ div, thu thập các thẻ p
            if description_div:
                description = ''
                paragraphs = description_div.find_all('p', {'class': 'zYHGB'})
                for p in paragraphs:
                    description += p.get_text().strip() + ' '
                description = description.strip()  # Xóa các khoảng trắng thừa ở đầu và cuối
            else:
                description = 'No description available'
                
            time.sleep(random.uniform(0.25, 0.28))


            # Tìm thông tin giải thưởng
            award = np.nan
            award_button = soup.find('button', {'class': 'ypcsE _S wSSLS'})

            if award_button:
                award_div = award_button.find('div', class_='biGQs _P pZUbB hmDzD')
                if award_div and "Travelers' Choice" in award_div.get_text():
                    award = "Travelers' Choice"
            time.sleep(random.uniform(0.25, 0.28))
            
            # Lấy thông tin số sao
            hotel_class = np.nan

            # Tìm thẻ <svg> có chứa 'aria-labelledby'
            svg_tag = soup.find('svg', {'aria-labelledby': True})

            if svg_tag:
                title_tag = svg_tag.find('title')
                
                # Kiểm tra nếu thẻ <title> tồn tại
                if title_tag:
                    hotel_class = title_tag.get_text()  
                else:
                    hotel_class = np.nan  
            else:
                hotel_class = np.nan  

                    
            time.sleep(random.uniform(0.25, 0.28))
            
            # Tìm thông tin ngôn ngữ sử dụng
            language = np.nan
            languages_spoken_label = soup.find('div', class_='MTDER f v Ci S2 H2 b Wf', string='Languages Spoken')

            if languages_spoken_label:
                language_div = languages_spoken_label.find_next_sibling('div')
                if language_div:
                    language = language_div.get_text()  
            
            time.sleep(random.uniform(0.25, 0.28))
            
            
            # Tìm thông tin kiểu khách sạn
            hotel_style_1 = np.nan
            hotel_style_2 = np.nan
            hotel_style_tag = soup.find('div', string="HOTEL STYLE")

            if hotel_style_tag:
                style_tags = hotel_style_tag.find_all_next('div', class_='CMiVw _R MC S4 _a H')
                
                if style_tags:
                    if len(style_tags) > 0:
                        hotel_style_1 = style_tags[0].get_text()  
                    if len(style_tags) > 1:
                        hotel_style_2 = style_tags[1].get_text()  
                        
            time.sleep(random.uniform(0.25, 0.28))
            
            # Di chuyển chuột đến giữa trang
            ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()

            # Cuộn xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            restaurant_count = np.nan
            attraction_count = np.nan
            res_attrac_divs = soup.find_all('div', class_='biGQs _P fiohW ngXxk')
            #res_attrac_divs = soup.find_all('div', class_='biGQs _P fiohW uuBRH')

            # Lọc các thẻ có nội dung chứa từ "Restaurants"
            for res_attrac_div in res_attrac_divs:
                if 'Restaurants' in res_attrac_div.text:
                    restaurant_count = res_attrac_div.text.split()[0]
                    # print(restaurant_count)
                
            for res_attrac_div in res_attrac_divs:
                if 'Attractions' in res_attrac_div.text:
                    attraction_count = res_attrac_div.text.split()[0]
            
            time.sleep(random.uniform(0.25, 0.28))
            
            # Tìm tất cả các thẻ <a> có class chứa thông tin nhà hàng và địa điểm du lịch
            all_links = soup.find_all('a', class_='BMQDV _F Gv wSSLS SwZTJ hNpWR FGwzt ukgoS')

            # Danh sách để lưu thông tin nhà hàng và địa điểm
            restaurants = []
            attractions = []

            # Tập hợp để theo dõi link đã xử lý (loại bỏ trùng lặp)
            processed_links = set()

            # Lặp qua từng thẻ để lấy thông tin
            for link in all_links:
                href = link['href']
                
                # Kiểm tra nếu link đã được xử lý
                if href in processed_links:
                    continue  # Bỏ qua nếu đã xử lý

                # Đánh dấu link đã xử lý
                processed_links.add(href)

                # Kiểm tra nếu là nhà hàng
                if "Restaurant_Review" in href:
                    res_name_div = link.find('div', class_='o W q')
                    if name_div:
                        res_name = res_name_div.text
                    else:
                        res_name = 'Tên không xác định'
                    restaurants.append((res_name, href))
                
                # Kiểm tra nếu là địa điểm du lịch
                elif "Attraction_Review" in href:
                    attrac_name_div = link.find('div', class_='o W q')
                    if name_div:
                        attrac_name = attrac_name_div.text
                    else:
                        attrac_name = 'Tên không xác định'
                    attractions.append((attrac_name, href))

            # Tạo đủ 4 biến cho nhà hàng và địa điểm du lịch (bao gồm cả tên và link)
            near_restaurant_1 = restaurants[0][0] if len(restaurants) > 0 else "Tên không xác định"
            link_near_restaurant_1 = 'https://www.tripadvisor.com' + restaurants[0][1] if len(restaurants) > 0 else ""
            near_restaurant_2 = restaurants[1][0] if len(restaurants) > 1 else "Tên không xác định"
            link_near_restaurant_2 = 'https://www.tripadvisor.com' + restaurants[1][1] if len(restaurants) > 1 else ""
            near_restaurant_3 = restaurants[2][0] if len(restaurants) > 2 else "Tên không xác định"
            link_near_restaurant_3 = 'https://www.tripadvisor.com' + restaurants[2][1] if len(restaurants) > 2 else ""
            near_restaurant_4 = restaurants[3][0] if len(restaurants) > 3 else "Tên không xác định"
            link_near_restaurant_4 = 'https://www.tripadvisor.com' + restaurants[3][1] if len(restaurants) > 3 else ""

            near_attraction_1 = attractions[0][0] if len(attractions) > 0 else "Tên không xác định"
            link_near_attraction_1 = 'https://www.tripadvisor.com' + attractions[0][1] if len(attractions) > 0 else ""
            near_attraction_2 = attractions[1][0] if len(attractions) > 1 else "Tên không xác định"
            link_near_attraction_2 = 'https://www.tripadvisor.com' + attractions[1][1] if len(attractions) > 1 else ""
            near_attraction_3 = attractions[2][0] if len(attractions) > 2 else "Tên không xác định"
            link_near_attraction_3 = 'https://www.tripadvisor.com' + attractions[2][1] if len(attractions) > 2 else ""
            near_attraction_4 = attractions[3][0] if len(attractions) > 3 else "Tên không xác định"
            link_near_attraction_4 = 'https://www.tripadvisor.com' + attractions[3][1] if len(attractions) > 3 else ""


            # time.sleep(random.uniform(0.25, 0.28))
            
            
            try:
                # Cố gắng tìm nút full view trong 10 giây
                full_view_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.QXsnf.s._U._S span.jVkLk'))
                )
                
                # Nếu nút full view tồn tại, di chuyển đến đó và click
                if full_view_button:
                    ActionChains(driver).move_to_element(full_view_button).click().perform()
                    time.sleep(0.5)  # Chờ cho hình ảnh hiển thị sau khi click

                    # Cập nhật lại BeautifulSoup sau khi trang tải thêm nội dung
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Tìm thẻ <img> có thuộc tính 'srcset'
                image_tag = soup.find('img', {'srcset': True})
                
                if image_tag:
                    # Lấy link ảnh từ thẻ <img>
                    image_url = image_tag['src']
                else:
                    image_url = np.nan

            except Exception as e:
                # Nếu không tìm thấy nút full view, thực hiện lấy ảnh trực tiếp từ thẻ <img>
                try:
                    # Tìm thẻ <img> có thuộc tính 'srcset' trong trang hiện tại
                    image_tag = soup.find('img', {'srcset': True})
                    
                    if image_tag:
                        image_url = image_tag['src']
                    else:
                        image_url = np.nan
                except Exception as img_error:
                    print(f"Image extraction failed: {img_error}")
                
            time.sleep(random.uniform(0.2, 0.25))
            
            # Ghi dữ liệu vào CSV
            writer.writerow({
                "Tourist Attraction": place,
                "Hotel Name": name,
                "Address": address,
                "Address Link": address_link,
                "Hotel Website": website,
                "Hotline": phone_number,
                "Ranking": ranking,
                "Reviewers": reviewers,
                "Rating": ratings,
                "Location rating": location_rating,
                "Cleanliness rating": cleanliness_rating,
                "Service rating": service_rating,
                "Value rating": value_rating,
                "Comment": comments,
                "Price": price,
                "URL": hotel_url,
                "Description": description,
                "Award": award,
                "Hotel Class": hotel_class,
                "Hotel Style 1": hotel_style_1,
                "Hotel Style 2": hotel_style_2,
                "Language Spoken": language,
                "Numbers of near Restaurants": restaurant_count,
                "Near Restaurant 1": near_restaurant_1,
                "Link Near Restaurant 1": link_near_restaurant_1,
                "Near Restaurant 2": near_restaurant_2,
                "Link Near Restaurant 2": link_near_restaurant_2,
                "Near Restaurant 3": near_restaurant_3,
                "Link Near Restaurant 3": link_near_restaurant_3,
                "Near Restaurant 4": near_restaurant_4,
                "Link Near Restaurant 4": link_near_restaurant_4,
                "Numbers of near Attractions": attraction_count,
                "Near Attraction 1": near_attraction_1,
                "Link Near Attraction 1": link_near_attraction_1,
                "Near Attraction 2": near_attraction_2,
                "Link Near Attraction 2": link_near_attraction_2,
                "Near Attraction 3": near_attraction_3,
                "Link Near Attraction 3": link_near_attraction_3,
                "Near Attraction 4": near_attraction_4,
                "Link Near Attraction 4": link_near_attraction_4,
                "Hotel Image": image_url
            })
            
            
            # Cập nhật thanh tiến độ sau mỗi khách sạn
            pbar.update(1)


In [19]:
hotel = pd.read_csv(f"Hotel_Data\hotel_{remove_accents_and_spaces(place)}.csv")
hotel

,Tourist Attraction,Hotel Name,Address,Address Link,Hotel Website,Hotline,Ranking,Reviewers,Rating,Location rating,...,Near Attraction 1,Link Near Attraction 1,Near Attraction 2,Link Near Attraction 2,Near Attraction 3,Link Near Attraction 3,Near Attraction 4,Link Near Attraction 4,Numbers of near Attractions,Hotel Image
0,Hà Nội,The Hanoi Club Hotel & Residences,"76 Yen Phu Street, Tay Ho, Hanoi 10000 Vietnam",NaN,https://www.tripadvisor.com,NaN,#7 of 20 hotels in Tay Ho,362,4.0,4.0,...,Chua Tran Quoc,https://www.tripadvisor.com/Attraction_Review-...,VietClimb,https://www.tripadvisor.com/Attraction_Review-...,West Lake,https://www.tripadvisor.com/Attraction_Review-...,Silver Soul Vietnam - Handmade Silver Jewelry,https://www.tripadvisor.com/Attraction_Review-...,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...
1,Hà Nội,The Oriental Jade Hotel,"92-94 Hang Trong Street, Hanoi Vietnam",NaN,https://www.tripadvisor.com,02439367777,#1 of 940 hotels in Hanoi,"3,703",5.0,5.0,...,Lake of the Restored Sword (Hoan Kiem Lake),https://www.tripadvisor.com/Attraction_Review-...,Thang Long Water Puppet,https://www.tripadvisor.com/Attraction_Review-...,An Handmade - H’Mong Hemp Fabric,https://www.tripadvisor.com/Attraction_Review-...,MasterTan,https://www.tripadvisor.com/Attraction_Review-...,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...
2,Hà Nội,Bespoke Trendy Hotel Hanoi,"12 Nguyen Quang Bich St., Hanoi 100000 Vietnam",NaN,NaN,NaN,#2 of 940 hotels in Hanoi,"5,118",5.0,4.9,...,An Handmade - H’Mong Hemp Fabric,https://www.tripadvisor.com/Attraction_Review-...,MasterTan,https://www.tripadvisor.com/Attraction_Review-...,Mountain's Color Shop,https://www.tripadvisor.com/Attraction_Review-...,An handmade Hang Manh - Ethnic Craft Station,https://www.tripadvisor.com/Attraction_Review-...,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...
3,Hà Nội,La Siesta Classic Ma May Hotel,"94 Ma May St., Hanoi Vietnam",NaN,NaN,NaN,#4 of 940 hotels in Hanoi,"6,509",5.0,4.9,...,Old Quarter,https://www.tripadvisor.com/Attraction_Review-...,Thang Long Water Puppet,https://www.tripadvisor.com/Attraction_Review-...,With Love From The Mountains,https://www.tripadvisor.com/Attraction_Review-...,An handmade Hang Manh - Ethnic Craft Station,https://www.tripadvisor.com/Attraction_Review-...,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...
4,Hà Nội,JM Marvel Hotel & Spa,"16 Hang Da Street Hoan Kiem District, Hanoi 10...",NaN,https://www.tripadvisor.com,02438238855,#9 of 940 hotels in Hanoi,"2,288",5.0,4.9,...,An Handmade - H’Mong Hemp Fabric,https://www.tripadvisor.com/Attraction_Review-...,MasterTan,https://www.tripadvisor.com/Attraction_Review-...,Mountain's Color Shop,https://www.tripadvisor.com/Attraction_Review-...,An handmade Hang Manh - Ethnic Craft Station,https://www.tripadvisor.com/Attraction_Review-...,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4627,Hà Nội,Vinhomes Ocean Park - LuxuryApartment S2.02 118A,"Vinhomes Ocean Park, Da Ton, Hanoi 100000 Vietnam",NaN,NaN,NaN,Write a review,"Vinhomes Ocean Park, Da Ton, Hanoi 100000 Vietnam",NaN,NaN,...,Quartzaholic - Handmade Jewelry & Gems,https://www.tripadvisor.com/Attraction_Review-...,VinKE & Aquarium Times City,https://www.tripadvisor.com/Attraction_Review-...,Aeon Mall Long Biên,https://www.tripadvisor.com/Attraction_Review-...,Viet Unique Golf,https://www.tripadvisor.com/Attraction_Review-...,NaN,https://dynamic-media-cdn.tripadvisor.com/medi...
4628,Hà Nội,Capsule Vinhomes Ocean Park,"Toa S2 08 Vinhomes Ocean Park, Da Ton, Hanoi V...",NaN,NaN,NaN,Write a review,"Toa S2 08 Vinhomes Ocean Park, Da Ton, Hanoi V...",NaN,NaN,...,Quartzaholic - Handmade Jewelry & Gems,https://www.tripadvisor.com/Attraction_Review-...,VinKE & Aquarium Times City,https://www.tripadvisor.com/Attraction_Review-...,Aeon Mall Long Biên,https://www.tripadvisor.com/Attraction_Review-...,Viet Unique Golf,https://www.tripadv